In [1]:
import re
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder


In [2]:
# Чтение данных из файла
with open('vin_ford_train.txt', 'r') as file:
    data = file.read()

In [3]:
# Паттерн для поиска VIN-кода и цены
pattern = re.compile(r'\[([A-Z0-9]{17})\:(\d+)\]')

In [4]:
# Находим все соответствия паттерну в файле
matches = pattern.findall(data)

In [5]:
# Создаем список кортежей, содержащих VIN-коды и цены
data_list = [(vin, int(price)) for vin, price in matches]

# Создаем датафрейм из списка
df = pd.DataFrame(data_list, columns=['VIN_code', 'price'])

# Выводим датафрейм
display(df)

,VIN_code,price
0,2FMDK3JC4BBA41556,12500
1,3FA6P0H75ER208976,14500
2,3FAHP08Z17R268380,1300
3,1FMCU0GX3DUC59421,15200
4,1FAHP2E89DG119368,11600
...,...,...
71880,1FTDX17W1VNB73890,1000
71881,1FADP3F23DL169365,12200
71882,3FA6P0HD2ER142859,16300
71883,1FAHP34N76W128169,1500


In [6]:
# Проверим валидность VIN-кодов в датафрейме 
# на соответствие длине и наличие недопустимых символов

# Паттерн для проверки длины и допустимых символов VIN-кода
pattern = re.compile(r'^[A-Z0-9]{17}$')

# Проверка каждого VIN-кода в столбце "VIN код"
valid_vins = []
invalid_vins = []
for vin in df['VIN_code']:
    if pattern.match(vin):
        valid_vins.append(vin)
    else:
        invalid_vins.append(vin)

In [7]:
# Разделяем VIN код на составляющие
df['WMI_country'] = df['VIN_code'].str.slice(0, 1)
df['WMI_manufacturer'] = df['VIN_code'].str.slice(1, 3)
df['VDS_model'] = df['VIN_code'].str.slice(3, 6)
df['VDS_body_type'] = df['VIN_code'].str.slice(6, 7)
df['VDS_engine_type'] = df['VIN_code'].str.slice(7, 8)
df['VDS_check_digit'] = df['VIN_code'].str.slice(8, 9)
df['year_of_manufacture'] = df['VIN_code'].str[9]
df['VIS_serial_number'] = df['VIN_code'].str.slice(10)

# Извлечение года производства из столбца 'year_of_manufacture'
# Таблица соответствия символов VIN кода и годов производства
# year_mapping = {
#     'A': 1980, 'B': 1981, 'C': 1982, 'D': 1983, 'E': 1984, 'F': 1985, 'G': 1986, 'H': 1987, 'J': 1988,
#     'K': 1989, 'L': 1990, 'M': 1991, 'N': 1992, 'P': 1993, 'R': 1994, 'S': 1995, 'T': 1996, 'V': 1997,
#     'W': 1998, 'X': 1999, 'Y': 2000, '1': 2001, '2': 2002, '3': 2003, '4': 2004, '5': 2005, '6': 2006,
#     '7': 2007, '8': 2008, '9': 2009
# }

# # Создаем новый столбец 'Manufacture_Year' с помощью метода .map()
# df['manufacture_year'] = df['year_of_manufacture'].map(year_mapping)


# Выводим результат
display(df)

,VIN_code,price,WMI_country,WMI_manufacturer,VDS_model,VDS_body_type,VDS_engine_type,VDS_check_digit,year_of_manufacture,VIS_serial_number
0,2FMDK3JC4BBA41556,12500,2,FM,DK3,J,C,4,B,BA41556
1,3FA6P0H75ER208976,14500,3,FA,6P0,H,7,5,E,R208976
2,3FAHP08Z17R268380,1300,3,FA,HP0,8,Z,1,7,R268380
3,1FMCU0GX3DUC59421,15200,1,FM,CU0,G,X,3,D,UC59421
4,1FAHP2E89DG119368,11600,1,FA,HP2,E,8,9,D,G119368
...,...,...,...,...,...,...,...,...,...,...
71880,1FTDX17W1VNB73890,1000,1,FT,DX1,7,W,1,V,NB73890
71881,1FADP3F23DL169365,12200,1,FA,DP3,F,2,3,D,L169365
71882,3FA6P0HD2ER142859,16300,3,FA,6P0,H,D,2,E,R142859
71883,1FAHP34N76W128169,1500,1,FA,HP3,4,N,7,6,W128169


In [8]:
df.drop(['VIN_code'], axis=1, inplace=True)


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71885 entries, 0 to 71884
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   price                71885 non-null  int64 
 1   WMI_country          71885 non-null  object
 2   WMI_manufacturer     71885 non-null  object
 3   VDS_model            71885 non-null  object
 4   VDS_body_type        71885 non-null  object
 5   VDS_engine_type      71885 non-null  object
 6   VDS_check_digit      71885 non-null  object
 7   year_of_manufacture  71885 non-null  object
 8   VIS_serial_number    71885 non-null  object
dtypes: int64(1), object(8)
memory usage: 4.9+ MB


In [10]:
# Преобразование категориальных признаков с помощью Label Encoding
label_encoder = LabelEncoder()
categorical_features = ['WMI_country', 'WMI_manufacturer', 'VDS_model', 
                        'VDS_body_type', 'VDS_engine_type', 
                        'VDS_check_digit', 'year_of_manufacture',
                        'VIS_serial_number']
for feature in categorical_features:
    df[feature] = label_encoder.fit_transform(df[feature])

In [11]:
# Разделяем данные на обучающий и тестовый наборы
X = df.drop(['price'], axis=1)  # Входные признаки
y = df['price']  # Целевая переменная

In [12]:
# Разделение данных на обучающий и тестовый наборы
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=12345)

# Проверка размеров наборов данных
print("Размер обучающего набора данных:", X_train.shape[0])
print("Размер тестового набора данных:", X_test.shape[0])

Размер обучающего набора данных: 57508
Размер тестового набора данных: 14377


In [13]:
# Инициализация модели
model = CatBoostRegressor(verbose=False)
# Обучение модели
model.fit(X_train, y_train)

In [14]:
# Предсказание на тестовом наборе данных
y_pred = model.predict(X_test)

In [15]:
# Вычисление метрик
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
rmse = np.sqrt(mse)  # Вычисление RMSE

# Организация метрик в таблицу
metrics_df = pd.DataFrame({
    'Mean Squared Error': [mse],
    'Mean Absolute Error': [mae],
    'R-squared': [r2],
    'Root Mean Squared Error': [rmse] 
})

print(metrics_df)

   Mean Squared Error  Mean Absolute Error  R-squared  Root Mean Squared Error
0        7.369522e+06          1900.942041   0.880199              2714.686358


In [16]:
# Получение важности признаков
feature_importance = model.feature_importances_

# Создание DataFrame с важностью признаков
importance_df = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance': feature_importance
})

# Сортировка признаков по важности
importance_df = importance_df.sort_values(by='Importance', ascending=False)

print(importance_df)

               Feature  Importance
3        VDS_body_type   22.270668
7    VIS_serial_number   21.374109
1     WMI_manufacturer   17.265552
6  year_of_manufacture   15.749956
2            VDS_model   14.031360
4      VDS_engine_type    8.821088
0          WMI_country    0.310198
5      VDS_check_digit    0.177069
